In [1]:
import pandas as pd
import numpy as np


In [2]:
orderReport_df = pd.read_excel('Company X - Order Report.xlsx')
pincodeZones_df = pd.read_excel('Company X - Pincode Zones.xlsx')
skuMaster_df = pd.read_excel('Company X - SKU Master.xlsx')


In [3]:
Invoice_df = pd.read_excel('Courier Company - Invoice.xlsx')
rates_df = pd.read_excel('Courier Company - Rates.xlsx')

In [4]:
alldf = [orderReport_df,pincodeZones_df,skuMaster_df,Invoice_df,rates_df]
for i in alldf:
    print(i.shape)
    print(i.size)
    print(i.head(20))
    print(i.isna().sum())
    print('-'*100)

(400, 3)
1200
    ExternOrderNo            SKU  Order Qty
0      2001827036  8904223818706        1.0
1      2001827036  8904223819093        1.0
2      2001827036  8904223819109        1.0
3      2001827036  8904223818430        1.0
4      2001827036  8904223819277        1.0
5      2001827036  GIFTBOX202002        1.0
6      2001827036  8904223818638        2.0
7      2001827036     SACHETS001        1.0
8      2001825261  8904223819024        4.0
9      2001825261  8904223819291        4.0
10     2001825261  8904223818638        3.0
11     2001825261  8904223818669        1.0
12     2001825261     SACHETS001        1.0
13     2001823564  8904223819291        2.0
14     2001823564  8904223819031        2.0
15     2001823564  8904223819024        2.0
16     2001822466  8904223819468        2.0
17     2001822466  8904223819291        8.0
18     2001821995  8904223819130        1.0
19     2001821995  8904223818706        1.0
ExternOrderNo    0
SKU              0
Order Qty        0
dtype

Preprocessing


In [5]:
rates_df.Zone.value_counts()

A    1
B    1
C    1
D    1
E    1
Name: Zone, dtype: int64

In [6]:
def zone(x):
    if x == 'A': return 'a'
    if x == 'B': return 'b'
    if x == 'C': return 'c'
    if x == 'D': return 'd'
    if x == 'E': return 'e'
    

In [7]:
rates_df.Zone = rates_df.Zone.apply(zone)

In [8]:
rates_df.head(10)

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,a,0.25,29.5,23.6,13.6,23.6
1,b,0.50,33.0,28.3,20.5,28.3
2,c,0.75,40.1,38.9,31.9,38.9
3,d,1.25,45.4,44.8,41.3,44.8
4,e,1.50,56.6,55.5,50.7,55.5


SKU stands for Stock Keeping Unit. A unique number is assigned to a product for internal tracking within eCommerce.
Generally, an eCommerce SKU number has information about the product, including the brand name, type of product, size, gender, model number, etc.

In [9]:
ordersku = orderReport_df.merge(skuMaster_df,on='SKU')


In [10]:
ordersku.head()

,ExternOrderNo,SKU,Order Qty,Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001821995,8904223818706,1.0,127
2,2001819252,8904223818706,1.0,127
3,2001816996,8904223818706,1.0,127
4,2001814580,8904223818706,1.0,127


In [11]:
zone_ddf =pd.merge(Invoice_df,pincodeZones_df,on=['Customer Pincode','Zone','Warehouse Pincode'],how='left')
zone_ddf

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4
...,...,...,...,...,...,...,...,...
123,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2
124,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7
125,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4
126,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0


In [12]:
print(pincodeZones_df.shape)
print(Invoice_df.shape)

(124, 3)
(124, 8)


In [13]:
border = pd.merge(zone_ddf,ordersku,how='inner',left_on='Order ID',right_on='ExternOrderNo')

In [14]:
print(ordersku.shape)
print(border.shape)
print(zone_ddf.shape)

(401, 4)
(415, 12)
(128, 8)


In [15]:
border.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),ExternOrderNo,SKU,Order Qty,Weight (g)
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,2001806232,8904223819147,2.0,240
1,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,2001806232,8904223818645,6.0,137
2,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,2001806273,8904223818706,1.0,127
3,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,2001806273,8904223818850,1.0,240
4,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,2001806273,8904223818942,1.0,133


In [16]:
border_slab = border.merge(rates_df,on='Zone')
border_slab.shape

(415, 17)

In [17]:
border_slab.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),ExternOrderNo,SKU,Order Qty,Weight (g),Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,2001806232,8904223819147,2.0,240,1.25,45.4,44.8,41.3,44.8
1,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,2001806232,8904223818645,6.0,137,1.25,45.4,44.8,41.3,44.8
2,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,2001806273,8904223818706,1.0,127,1.25,45.4,44.8,41.3,44.8
3,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,2001806273,8904223818850,1.0,240,1.25,45.4,44.8,41.3,44.8
4,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,2001806273,8904223818942,1.0,133,1.25,45.4,44.8,41.3,44.8


In [18]:
border_slab.columns

Index(['AWB Code', 'Order ID', 'Charged Weight', 'Warehouse Pincode',
       'Customer Pincode', 'Zone', 'Type of Shipment', 'Billing Amount (Rs.)',
       'ExternOrderNo', 'SKU', 'Order Qty', 'Weight (g)', 'Weight Slabs',
       'Forward Fixed Charge', 'Forward Additional Weight Slab Charge',
       'RTO Fixed Charge', 'RTO Additional Weight Slab Charge'],
      dtype='object')

In [19]:
border_slab['Type of Shipment'].value_counts()

Forward charges            365
Forward and RTO charges     50
Name: Type of Shipment, dtype: int64

In [20]:
border_slab['Weight (g)'] = border_slab['Weight (g)'].apply(lambda x: x/1000)

In [21]:
border_slab = border_slab.rename(columns={'Weight (g)':'Weight (KG)'})

In [22]:
border_slab['Total Weight of SKU'] = border_slab['Weight (KG)']*border_slab['Order Qty']

In [23]:
border_slab.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),ExternOrderNo,SKU,Order Qty,Weight (KG),Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,Total Weight of SKU
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,2001806232,8904223819147,2.0,0.240,1.25,45.4,44.8,41.3,44.8,0.480
1,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,2001806232,8904223818645,6.0,0.137,1.25,45.4,44.8,41.3,44.8,0.822
2,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,2001806273,8904223818706,1.0,0.127,1.25,45.4,44.8,41.3,44.8,0.127
3,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,2001806273,8904223818850,1.0,0.240,1.25,45.4,44.8,41.3,44.8,0.240
4,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,2001806273,8904223818942,1.0,0.133,1.25,45.4,44.8,41.3,44.8,0.133


In [24]:
grouped_df1 = border_slab.groupby(['Zone','Order ID']).agg({'Total Weight of SKU': 'sum', 
                                         'AWB Code': 'first',
                                         'Charged Weight': 'first',
                                         'Warehouse Pincode': 'first',
                                         'Customer Pincode': 'first',
                                         'Type of Shipment': 'first',
                                         'Billing Amount (Rs.)': 'first',
                                         'Weight Slabs': 'first',
                                         'Forward Fixed Charge': 'first',
                                         'Forward Additional Weight Slab Charge': 'first',
                                         'RTO Fixed Charge': 'first',
                                         'RTO Additional Weight Slab Charge': 'first'})


In [25]:
grouped_df1.head(10).sort_values(by='Total Weight of SKU',ascending=False)

Total Weight of SKU       AWB Code  Charged Weight  \
Zone Order ID                                                         
b    2001807058                2.336  1091117323112            1.15   
     2001806575                1.000  1091117225016            0.68   
     2001806616                0.963  1091117225484            1.08   
     2001806885                0.840  1091117229776            1.00   
     2001806801                0.722  1091117227816            1.35   
     2001806458                0.700  1091117223244            1.00   
     2001806776                0.611  1091117227573            2.86   
     2001806233                0.245  1091117222135            0.78   
     2001806210                0.220  1091117221940            2.92   
     2001806547                0.127  1091117224611            1.00   

                 Warehouse Pincode  Customer Pincode Type of Shipment  \
Zone Order ID                                                           
b    2001807058             121003            140301  Forward charges   
     2001806575             121003            248001  Forward charges   
     2001806616             121003            144001  Forward charges   
     2001806885             121003            208019  Forward charges   
     2001806801             121003            248001  Forward charges   
     2001806458             121003            143001  Forward charges   
     2001806776             121003            226004  Forward charges   
     2001806233             121003            263139  Forward charges   
     2001806210             121003            140604  Forward charges   
     2001806547             121003            283102  Forward charges   

                 Billing Amount (Rs.)  Weight Slabs  Forward Fixed Charge  \
Zone Order ID                                                               
b    2001807058                  89.6           0.5                  33.0   
     2001806575                  61.3           0.5                  33.0   
     2001806616                  89.6           0.5                  33.0   
     2001806885                  61.3           0.5                  33.0   
     2001806801                  89.6           0.5                  33.0   
     2001806458                  61.3           0.5                  33.0   
     2001806776                 174.5           0.5                  33.0   
     2001806233                  61.3           0.5                  33.0   
     2001806210                 174.5           0.5                  33.0   
     2001806547                  61.3           0.5                  33.0   

                 Forward Additional Weight Slab Charge  RTO Fixed Charge  \
Zone Order ID                                                              
b    2001807058                                   28.3              20.5   
     2001806575                                   28.3              20.5   
     2001806616                                   28.3              20.5   
     2001806885                                   28.3              20.5   
     2001806801                                   28.3              20.5   
     2001806458                                   28.3              20.5   
     2001806776                                   28.3              20.5   
     2001806233                                   28.3              20.5   
     2001806210                                   28.3              20.5   
     2001806547                                   28.3              20.5   

                 RTO Additional Weight Slab Charge  
Zone Order ID                                       
b    2001807058                               28.3  
     2001806575                               28.3  
     2001806616                               28.3  
     2001806885                               28.3  
     2001806801                               28.3  
     2001806458                               28.3  
     2001806776          

In [26]:
grouped_df1.shape

(124, 12)

In [27]:
grouped_df1.columns

Index(['Total Weight of SKU', 'AWB Code', 'Charged Weight',
       'Warehouse Pincode', 'Customer Pincode', 'Type of Shipment',
       'Billing Amount (Rs.)', 'Weight Slabs', 'Forward Fixed Charge',
       'Forward Additional Weight Slab Charge', 'RTO Fixed Charge',
       'RTO Additional Weight Slab Charge'],
      dtype='object')

In [28]:
def charges(x): # rw
    total_weight = x['Charged Weight']
    weight_slab = x['Weight Slabs']
    fwd_fixed_charge = x['Forward Fixed Charge']
    fwd_additional_charge = x['Forward Additional Weight Slab Charge']
    rto_fixed_charge = x['RTO Fixed Charge']
    rto_additional_charge = x['RTO Additional Weight Slab Charge']
    type_of_shipment = x['Type of Shipment']

    #no of slabs
    no_of_slabs = np.ceil(total_weight/weight_slab)


    if type_of_shipment == 'Forward charges':
        total_charges = fwd_fixed_charge + (no_of_slabs-1) * fwd_additional_charge
    elif type_of_shipment == 'Forward and RTO charges':
        fwd_charges = fwd_fixed_charge + (no_of_slabs - 1) * fwd_additional_charge
        rto_charges = rto_fixed_charge + (no_of_slabs - 1) * rto_additional_charge
        total_charges = fwd_charges + rto_charges
    return total_charges



In [7]:
status = ['inprogess','success','fail','success']
for i in status:
    if i == 'success':
        print(i)
        break

success


In [29]:
grouped_df1['Expected Charge as per X (Rs.)'] = grouped_df1.apply(charges,axis= 1)

In [30]:
grouped_df1.head(10)

Total Weight of SKU       AWB Code  Charged Weight  \
Zone Order ID                                                         
b    2001806210                0.220  1091117221940            2.92   
     2001806233                0.245  1091117222135            0.78   
     2001806458                0.700  1091117223244            1.00   
     2001806547                0.127  1091117224611            1.00   
     2001806575                1.000  1091117225016            0.68   
     2001806616                0.963  1091117225484            1.08   
     2001806776                0.611  1091117227573            2.86   
     2001806801                0.722  1091117227816            1.35   
     2001806885                0.840  1091117229776            1.00   
     2001807058                2.336  1091117323112            1.15   

                 Warehouse Pincode  Customer Pincode Type of Shipment  \
Zone Order ID                                                           
b    2001806210             121003            140604  Forward charges   
     2001806233             121003            263139  Forward charges   
     2001806458             121003            143001  Forward charges   
     2001806547             121003            283102  Forward charges   
     2001806575             121003            248001  Forward charges   
     2001806616             121003            144001  Forward charges   
     2001806776             121003            226004  Forward charges   
     2001806801             121003            248001  Forward charges   
     2001806885             121003            208019  Forward charges   
     2001807058             121003            140301  Forward charges   

                 Billing Amount (Rs.)  Weight Slabs  Forward Fixed Charge  \
Zone Order ID                                                               
b    2001806210                 174.5           0.5                  33.0   
     2001806233                  61.3           0.5                  33.0   
     2001806458                  61.3           0.5                  33.0   
     2001806547                  61.3           0.5                  33.0   
     2001806575                  61.3           0.5                  33.0   
     2001806616                  89.6           0.5                  33.0   
     2001806776                 174.5           0.5                  33.0   
     2001806801                  89.6           0.5                  33.0   
     2001806885                  61.3           0.5                  33.0   
     2001807058                  89.6           0.5                  33.0   

                 Forward Additional Weight Slab Charge  RTO Fixed Charge  \
Zone Order ID                                                              
b    2001806210                                   28.3              20.5   
     2001806233                                   28.3              20.5   
     2001806458                                   28.3              20.5   
     2001806547                                   28.3              20.5   
     2001806575                                   28.3              20.5   
     2001806616                                   28.3              20.5   
     2001806776                                   28.3              20.5   
     2001806801                                   28.3              20.5   
     2001806885                                   28.3              20.5   
     2001807058                                   28.3              20.5   

                 RTO Additional Weight Slab Charge  \
Zone Order ID                                        
b    2001806210                               28.3   
     2001806233                               28.3   
     2001806458                               28.3   
     2001806547                               28.3   
     2001806575                               28.3   
     2001806616                               28.3   
     2001806776  

In [31]:
grouped_df1['Difference expected charge and billed charge'] = grouped_df1['Expected Charge as per X (Rs.)'] - grouped_df1['Billing Amount (Rs.)']

In [32]:
grouped_df1.head(10).sort_values(by='Difference expected charge and billed charge',ascending=False)

Total Weight of SKU       AWB Code  Charged Weight  \
Zone Order ID                                                         
b    2001806210                0.220  1091117221940            2.92   
     2001806233                0.245  1091117222135            0.78   
     2001806458                0.700  1091117223244            1.00   
     2001806547                0.127  1091117224611            1.00   
     2001806575                1.000  1091117225016            0.68   
     2001806616                0.963  1091117225484            1.08   
     2001806776                0.611  1091117227573            2.86   
     2001806801                0.722  1091117227816            1.35   
     2001806885                0.840  1091117229776            1.00   
     2001807058                2.336  1091117323112            1.15   

                 Warehouse Pincode  Customer Pincode Type of Shipment  \
Zone Order ID                                                           
b    2001806210             121003            140604  Forward charges   
     2001806233             121003            263139  Forward charges   
     2001806458             121003            143001  Forward charges   
     2001806547             121003            283102  Forward charges   
     2001806575             121003            248001  Forward charges   
     2001806616             121003            144001  Forward charges   
     2001806776             121003            226004  Forward charges   
     2001806801             121003            248001  Forward charges   
     2001806885             121003            208019  Forward charges   
     2001807058             121003            140301  Forward charges   

                 Billing Amount (Rs.)  Weight Slabs  Forward Fixed Charge  \
Zone Order ID                                                               
b    2001806210                 174.5           0.5                  33.0   
     2001806233                  61.3           0.5                  33.0   
     2001806458                  61.3           0.5                  33.0   
     2001806547                  61.3           0.5                  33.0   
     2001806575                  61.3           0.5                  33.0   
     2001806616                  89.6           0.5                  33.0   
     2001806776                 174.5           0.5                  33.0   
     2001806801                  89.6           0.5                  33.0   
     2001806885                  61.3           0.5                  33.0   
     2001807058                  89.6           0.5                  33.0   

                 Forward Additional Weight Slab Charge  RTO Fixed Charge  \
Zone Order ID                                                              
b    2001806210                                   28.3              20.5   
     2001806233                                   28.3              20.5   
     2001806458                                   28.3              20.5   
     2001806547                                   28.3              20.5   
     2001806575                                   28.3              20.5   
     2001806616                                   28.3              20.5   
     2001806776                                   28.3              20.5   
     2001806801                                   28.3              20.5   
     2001806885                                   28.3              20.5   
     2001807058                                   28.3              20.5   

                 RTO Additional Weight Slab Charge  \
Zone Order ID                                        
b    2001806210                               28.3   
     2001806233                               28.3   
     2001806458                               28.3   
     2001806547                               28.3   
     2001806575                               28.3   
     2001806616                               28.3   
     2001806776  

In [33]:
grouped_df2 = grouped_df1.reset_index()

In [34]:
grouped_df2.columns

Index(['Zone', 'Order ID', 'Total Weight of SKU', 'AWB Code', 'Charged Weight',
       'Warehouse Pincode', 'Customer Pincode', 'Type of Shipment',
       'Billing Amount (Rs.)', 'Weight Slabs', 'Forward Fixed Charge',
       'Forward Additional Weight Slab Charge', 'RTO Fixed Charge',
       'RTO Additional Weight Slab Charge', 'Expected Charge as per X (Rs.)',
       'Difference expected charge and billed charge'],
      dtype='object')

In [35]:
grouped_df2 = grouped_df2[['Order ID','AWB Code','Total Weight of SKU','Weight Slabs','Charged Weight', 'Forward Fixed Charge', 'Forward Additional Weight Slab Charge', 'RTO Fixed Charge', 'RTO Additional Weight Slab Charge', 'Zone','Expected Charge as per X (Rs.)','Billing Amount (Rs.)','Difference expected charge and billed charge']]

In [36]:
grouped_df2.head()

,Order ID,AWB Code,Total Weight of SKU,Weight Slabs,Charged Weight,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,Zone,Expected Charge as per X (Rs.),Billing Amount (Rs.),Difference expected charge and billed charge
0,2001806210,1091117221940,0.220,0.5,2.92,33.0,28.3,20.5,28.3,b,174.5,174.5,0.0
1,2001806233,1091117222135,0.245,0.5,0.78,33.0,28.3,20.5,28.3,b,61.3,61.3,0.0
2,2001806458,1091117223244,0.700,0.5,1.00,33.0,28.3,20.5,28.3,b,61.3,61.3,0.0
3,2001806547,1091117224611,0.127,0.5,1.00,33.0,28.3,20.5,28.3,b,61.3,61.3,0.0
4,2001806575,1091117225016,1.000,0.5,0.68,33.0,28.3,20.5,28.3,b,61.3,61.3,0.0


In [37]:
grouped_df2 = grouped_df2.rename(columns={'Total Weight of SKU':'Total weight as per X (KG)', \
                                          'Weight Slabs':'Weight slab as per X (KG)', \
                                                 'Charged Weight':'Total weight as per Courier Company (KG)',\
                                                        'Zone':'Delivery zone as per X',\
                                                               'Billing Amount (Rs.)':'Charges Billed by Courier Company (Rs.)'})

In [38]:
# Output Data 1
grouped_df2.to_excel('Outputdata1_Order Level.xlsx')

In [105]:
def amount(x):
    if x == 0: return 'Correctly charged'
    if x < 0: return 'Undercharged'
    if x > 0: return 'Overcharged'

In [106]:
df2 = grouped_df1

In [107]:
df2['amount type'] = grouped_df1['Difference expected charge and billed charge'].apply(amount)

In [108]:
df2.head()

Total Weight of SKU       AWB Code  Charged Weight  \
Zone Order ID                                                         
b    2001806210                0.220  1091117221940            2.92   
     2001806233                0.245  1091117222135            0.78   
     2001806458                0.700  1091117223244            1.00   
     2001806547                0.127  1091117224611            1.00   
     2001806575                1.000  1091117225016            0.68   

                 Warehouse Pincode  Customer Pincode Type of Shipment  \
Zone Order ID                                                           
b    2001806210             121003            140604  Forward charges   
     2001806233             121003            263139  Forward charges   
     2001806458             121003            143001  Forward charges   
     2001806547             121003            283102  Forward charges   
     2001806575             121003            248001  Forward charges   

                 Billing Amount (Rs.)  Weight Slabs  Forward Fixed Charge  \
Zone Order ID                                                               
b    2001806210                 174.5           0.5                  33.0   
     2001806233                  61.3           0.5                  33.0   
     2001806458                  61.3           0.5                  33.0   
     2001806547                  61.3           0.5                  33.0   
     2001806575                  61.3           0.5                  33.0   

                 Forward Additional Weight Slab Charge  RTO Fixed Charge  \
Zone Order ID                                                              
b    2001806210                                   28.3              20.5   
     2001806233                                   28.3              20.5   
     2001806458                                   28.3              20.5   
     2001806547                                   28.3              20.5   
     2001806575                                   28.3              20.5   

                 RTO Additional Weight Slab Charge  \
Zone Order ID                                        
b    2001806210                               28.3   
     2001806233                               28.3   
     2001806458                               28.3   
     2001806547                               28.3   
     2001806575                               28.3   

                 Expected Charge as per X (Rs.)  \
Zone Order ID                                     
b    2001806210                           174.5   
     2001806233                            61.3   
     2001806458                            61.3   
     2001806547                            61.3   
     2001806575                            61.3   

                 Difference expected charge and billed charge  \
Zone Order ID                                                   
b    2001806210                                           0.0   
     2001806233                                           0.0   
     2001806458                                           0.0   
     2001806547                                           0.0   
     2001806575                                           0.0   

                       amount type  
Zone Order ID                       
b    2001806210  Correctly charged  
     2001806233  Correctly charged  
     2001806458  Correctly charged  
     2001806547  Correctly charged  
     2001806575  Correctly charged

In [109]:
df2['amount type'].value_counts()

Undercharged         83
Correctly charged    38
Overcharged           3
Name: amount type, dtype: int64

In [110]:
df3 = df2.groupby('amount type').agg({'Difference expected charge and billed charge':'sum','amount type':'count'})

In [111]:
df3

,Difference expected charge and billed charge,amount type
amount type,,
Correctly charged,0.0,38
Overcharged,23.4,3
Undercharged,-5529.8,83


In [112]:
df3 = df3.rename(columns={'amount type':'Count','Difference expected charge and billed charge':'Amount (Rs.)'})

In [113]:
df3 = df3.reset_index()

In [114]:
df3

,amount type,Amount (Rs.),Count
0,Correctly charged,0.0,38
1,Overcharged,23.4,3
2,Undercharged,-5529.8,83


In [115]:
sum_amount = df2.loc[df2['amount type'] == 'Correctly charged', 'Billing Amount (Rs.)'].sum()
sum_amount

2442.9000000000005

In [116]:
df3.loc[df3['amount type'] == 'Correctly charged', 'Amount (Rs.)'] = sum_amount

In [117]:
df3

,amount type,Amount (Rs.),Count
0,Correctly charged,2442.9,38
1,Overcharged,23.4,3
2,Undercharged,-5529.8,83


In [118]:
df3.info

<bound method DataFrame.info of          amount type  Amount (Rs.)  Count
0  Correctly charged        2442.9     38
1        Overcharged          23.4      3
2       Undercharged       -5529.8     83>

In [119]:
df3 = df3.rename(columns={'amount type':'Order summary'})
df3['Order summary'] = 'Total orders where X has been' + df3['Order summary']

In [120]:
df3 = df3[['Order summary', 'Count', 'Amount (Rs.)']]

In [121]:
df3.to_excel('Output_Data2_ Summary Table.xlsx')